In [1]:
# ! pip install faiss-cpu gradio transformers torchvision torch --quiet
! pip install faiss-cpu gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 32.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 113.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.7 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import torch
import faiss
from PIL import Image
import gradio as gr
from torchvision import transforms
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel

2025-06-25 07:20:31.504073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750836031.737648      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750836031.804919      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# --------- 1. Global Configuration ---------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMAGE_FOLDER = "Dataset"  # Folder containing images to index
INDEX_FILE = "output/faiss_index.bin"
FEATURES_FILE = "outputfeatures.npy"
FILENAMES_FILE = "output/filenames.npy"
TOP_K = 5  # Number of results to return


# --------- 2. CLIP Model Initialization ---------
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(DEVICE)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [4]:
# --------- 3. Feature Extraction Function ---------
def extract_clip_features(image: Image.Image):
    inputs = clip_processor(images=image, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
        image_features /= image_features.norm(p=2, dim=-1, keepdim=True)  # L2 normalize
    return image_features.cpu().numpy().astype("float32")

print('extract_clip_features function was defined successfully')

extract_clip_features function was defined successfully


In [5]:
# --------- 4. Build Index from Dataset ---------
def build_index(image_folder: str):
    features = []
    filenames = []
    counter = 0
    
    print("Extracting features from dataset...")
    for filename in tqdm(os.listdir(image_folder)):
        if filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
            path = os.path.join(image_folder, filename)
            try:
                image = Image.open(path).convert("RGB")
                feature = extract_clip_features(image)
                features.append(feature)
                filenames.append(path)
            except Exception as e:
                counter += 1

    features = np.vstack(features)
    index = faiss.IndexFlatL2(features.shape[1])
    index.add(features)

    # Save index and metadata
    faiss.write_index(index, INDEX_FILE)
    np.save(FEATURES_FILE, features)
    np.save(FILENAMES_FILE, np.array(filenames))

    print(f"Indexing complete. {counter} files were passed")
    return index, filenames

print('build_index function was defined successfully')

build_index function was defined successfully


In [6]:
# --------- 5. Load or Build FAISS Index ---------
def load_or_build_index():
    if os.path.exists(INDEX_FILE) and os.path.exists(FILENAMES_FILE):
        print("Loading saved index...")
        index = faiss.read_index(INDEX_FILE)
        filenames = np.load(FILENAMES_FILE, allow_pickle=True)
        print("Loading saved index completed")
    else:
        print("No saved index found. Building new index...")
        index, filenames = build_index(IMAGE_FOLDER)
    return index, filenames

print('load_or_build_index function was defined successfully')

load_or_build_index function was defined successfully


In [7]:
# --------- 6. Search Function ---------
def search_similar_images(query_image: Image.Image):
    query_vector = extract_clip_features(query_image)
    distances, indices = index.search(query_vector, TOP_K)
    results = [filenames[i] for i in indices[0]]
    return results

print('search_similar_images function was defined successfully')

search_similar_images function was defined successfully


In [8]:
# --------- 7. Gradio Interface ---------
def gradio_search_interface(input_image):
    results = search_similar_images(input_image)
    return [Image.open(img_path) for img_path in results]

index, filenames = load_or_build_index()

interface = gr.Interface(
    fn=gradio_search_interface,
    inputs=gr.Image(type="pil", label="Upload a Query Image"),
    outputs=[gr.Image(label=f"Result {i+1}") for i in range(TOP_K)],
    title="CLIP-Based Image Search Engine",
    description="Upload an image to find visually and semantically similar images using CLIP and FAISS.",
)

No saved index found. Building new index...
Extracting features from dataset...


100%|██████████| 24999/24999 [06:47<00:00, 61.30it/s]


Indexing complete.


In [9]:
interface.launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://defc1dc1670fea1fba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
